# Creating my own RQA function using base python

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix

### Need to think how to get from a time series to a matrix e.g.

In [2]:
ts = [1,2,3]
ts

[1, 2, 3]

In [3]:
ts_matrix = [[1,2,3],
             [2,3,1],
             [3,2,1]]
np.asarray(ts_matrix)

array([[1, 2, 3],
       [2, 3, 1],
       [3, 2, 1]])

### Initialise a list of lists

In [4]:
x = [[2,1,3],[0,1,1],[2,2,3]]

### Convert the list of lists to an array

In [5]:
x = np.asarray(x)
x

array([[2, 1, 3],
       [0, 1, 1],
       [2, 2, 3]])

### Original function from scipy - source code can be found [here](https://github.com/scipy/scipy/blob/v1.3.0/scipy/spatial/kdtree.py#L936-L987)

In [6]:
# this is the original function from scipy to confirm our reconstruction is correct
distance_matrix(x,x, p=2)

array([[0.        , 2.82842712, 1.        ],
       [2.82842712, 0.        , 3.        ],
       [1.        , 3.        , 0.        ]])

### The Minkowski distance or Minkowski metric is a generalisation of multiple distance measures, this arises from the order:

- λ = 1 is the Manhattan distance. Synonyms are L1-Norm, Taxicab or City-Block distance. For two vectors of ranked ordinal variables, the Manhattan distance is sometimes called Foot-ruler distance.
- λ = 2 is the Euclidean distance. Synonyms are L2-Norm or Ruler distance. For two vectors of ranked ordinal variables, the Euclidean distance is sometimes called Spear-man distance.
- λ = ∞ is the Chebyshev distance. Synonyms are Lmax-Norm or Chessboard distance.

<img src="Minkowski_distance_formula.png">

In [7]:
def minkowski_distance(x, y, p=2):
    return np.sum(np.abs(y-x)**p, axis=-1)**(1./p)

In [8]:
# def distance_matrix_jms(x, y, p=2):
#     m, k = x.shape
#     n, kk = y.shape
    
#     result = np.empty((m,n),dtype=float)
#     if m < n:
#         for i in range(m):
#             result[i,:] = minkowski_distance(x[i],y,p)
#     else:
#         for j in range(n):
#             result[:,j] = minkowski_distance(x,y[j],p)
#         return result

In [9]:
# simplest version, not sure what will happen if we try to add lags though :3
def distance_matrix_jms(x, y, p=2):
    m, k = x.shape
    n, kk = y.shape
    
    result = np.empty((m,n),dtype=float)
    for i in range(m):
        result[i,:] = minkowski_distance(x[i],y,p)
    return result

### James version of the distance_matrix function - it currently only accepts numpy arrays

In [10]:
rm = distance_matrix_jms(x, x, p=2)
rm

array([[0.        , 2.82842712, 1.        ],
       [2.82842712, 0.        , 3.        ],
       [1.        , 3.        , 0.        ]])

### Remove the diagonal from the matrix - they will always be zero if it is auto - reccurrence

In [11]:
def remove_diag(x):
    x_no_diag = np.ndarray.flatten(x)
    #print(x_no_diag)
    x_no_diag = np.delete(x_no_diag, range(0, len(x_no_diag), len(x) + 1), 0)
    #print(x_no_diag)
    x_no_diag = x_no_diag.reshape(len(x), len(x) - 1)
    #print(x_no_diag)
    return x_no_diag

In [12]:
rm_no_diag = remove_diag(rm)
rm_no_diag

array([[2.82842712, 1.        ],
       [2.82842712, 3.        ],
       [1.        , 3.        ]])

### Threshold the matrix at a certain radius i.e. make any value below a certain number 0, and any number above 1

In [13]:
threshold = 1.5
recmat = rm_no_diag
thresh_mat_all = []
thresh_mat = []
for i in recmat:
    for j in range(len(i)):
        if i[j] < threshold:
            thresh_mat.append(1)
        else:
            thresh_mat.append(0)
thresh_mat_all.append(thresh_mat)
thresh_mat_all

[[0, 1, 0, 0, 1, 0]]

In [14]:
# create chunks function which returns n-chunks of a list
def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [15]:
# create flatten function to use when you have lists within lists
flatten = lambda l: [item for sublist in l for item in sublist]

In [16]:
thresh_mat_arr = np.asarray(list(chunks(flatten(thresh_mat_all), 2)))
thresh_mat_arr

array([[0, 1],
       [0, 0],
       [1, 0]])

In [17]:
# the amount of recurrent points in the distance matrix
RP_N = np.sum(thresh_mat_arr)
RP_N

2

In [18]:
# the recurrence rate in the distance matrix
RR = RP_N/len(flatten(thresh_mat_arr))
RR

0.3333333333333333

In [19]:
# Casnet (R) version of the RQA measures - RR, Determinism, and Laminarity
#   #Total nr. recurrent points
#   RP_N <- Matrix::nnzero(RM, na.counted = FALSE)

#   minDiag <- 0
#   if(Matrix::isSymmetric(Matrix::unname(RM))){
#     if(all(Matrix::diag(RM)==1)){
#       minDiag <- length(Matrix::diag(RM))
#       }
#   }

#   RP_N <- RP_N-minDiag

#   #Proportion recurrence / Recurrence Rate
#   RR <- RP_N/recmatsize

#   if(length(RR)==0){RR<-0}

#   if(RR==1){
#     warning("Everything is recurring!\nReturning empty vector")
#     return(crqa_rp_empty())
#   }


#   #Get line segments
#   lineSegments <- rp_lineDist(RM,
#                               DLmin = DLmin, DLmax = DLmax,
#                               VLmin = VLmin, VLmax = VLmax,
#                               HLmin = HLmin, HLmax = HLmax,
#                               theiler = theiler, AUTO = AUTO)

#   dlines <- lineSegments$diagonals.dist
#   vlines <- lineSegments$verticals.dist
#   hlines <- lineSegments$horizontals.dist

#   #Frequency tables of line lengths
#   freq_dl <- table(dlines)
#   freq_vl <- table(vlines)
#   freq_hl <- table(hlines)

#   freqvec_dl <- as.numeric(names(freq_dl))
#   freqvec_vl <- as.numeric(names(freq_vl))
#   freqvec_hl <- as.numeric(names(freq_hl))

#   # Number of lines
#   N_dl <- sum(freq_dl, na.rm = TRUE)
#   N_vl <- sum(freq_vl, na.rm = TRUE)
#   N_hl <- sum(freq_hl, na.rm = TRUE)

#   #Number of recurrent points on diagonal, vertical and horizontal lines
#   N_dlp <- sum(freqvec_dl*freq_dl, na.rm = TRUE)
#   N_vlp <- sum(freqvec_vl*freq_vl, na.rm = TRUE)
#   N_hlp <- sum(freqvec_hl*freq_hl, na.rm = TRUE)

#   #Determinism / Horizontal and Vertical Laminarity
#   DET    <- N_dlp/RP_N
#   LAM_vl <- N_vlp/RP_N
#   LAM_hl <- N_hlp/RP_N